In [5]:
# train_resnet3d.ipynb starter (Python script format)

import os
import torch
import torch.nn as nn
import torch.optim as optim
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from My_3DResNet_Model import ResNet3D

In [ ]:
# Dataset for 3D classification from .nii.gz files
class MRIDataset(Dataset):
    def __init__(self, image_paths, labels_df, label_column="facial_features_present", transform=None):
        self.image_paths = image_paths
        self.labels_df = labels_df
        self.label_column = label_column
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __tensorconversion__(self, idx):
        img_path = self.image_paths[idx]
        img = nib.load(img_path).get_fdata().astype(np.float32)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)  # normalize
        img = np.expand_dims(img, axis=0)  # [C, D, H, W]

        filename = os.path.basename(img_path).strip().lower()
        if filename not in self.labels_df.index:
            raise KeyError(f"Filename '{filename}' not found in labels DataFrame index. Sample index: {self.labels_df.index[:5].tolist()}")
        label_str = self.labels_df.loc[filename][self.label_column]
        label = 1.0 if str(label_str).strip().lower() in ['yes', '1', 'true'] else 0.0

        return torch.tensor(img), torch.tensor(label, dtype=torch.float32)


In [7]:
# Paths and file loading
image_dir = "../dataset/files"
label_csv = "../labels.csv"
labels_df = pd.read_csv(label_csv)

# Print column names and head for debugging
# print("CSV Columns:", labels_df.columns.tolist())
# print(labels_df.head())

# Normalize extensions and ensure matching index format
labels_df.iloc[:, 0] = labels_df.iloc[:, 0].astype(str).apply(lambda x: x.strip().lower().removesuffix('.nii').removesuffix('.gz') + '.nii.gz')
labels_df.set_index(labels_df.columns[0], inplace=True)

image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".nii.gz")])

# Dataset and DataLoader with validation and test split
from sklearn.model_selection import train_test_split

train_val_files, test_files = train_test_split(image_files, test_size=0.1, random_state=42)
train_files, val_files = train_test_split(train_val_files, test_size=0.2, random_state=42)

# For label classifier
label_column = "Recognizable-Facial-Feature" # "Recognizable-Facial-Feature", "Brain-Feature-Loss"

train_dataset = MRIDataset(train_files, labels_df, label_column=label_column)
val_dataset = MRIDataset(val_files, labels_df, label_column=label_column)
test_dataset = MRIDataset(test_files, labels_df, label_column=label_column)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [8]:
# Model setup
model = ResNet3D().cuda()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [31]:
# Training loop (multiple epochs with validation + tracking)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

n_epochs = 5
best_f1 = 0.0
train_history = []
val_history = []

for epoch in range(n_epochs):
    model.train()
    predictions = []
    targets = []
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = (outputs > 0.5).float()
        predictions.extend(preds.cpu().numpy())
        targets.extend(labels.cpu().numpy())

        running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}", flush=True)

    train_acc = accuracy_score(targets, predictions)
    train_prec = precision_score(targets, predictions, zero_division=0)
    train_rec = recall_score(targets, predictions, zero_division=0)
    train_f1 = f1_score(targets, predictions, zero_division=0)
    train_history.append((train_acc, train_prec, train_rec, train_f1))

    # Validation metrics
    model.eval()
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            preds = (outputs > 0.5).float()
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_targets, val_preds)
    val_prec = precision_score(val_targets, val_preds, zero_division=0)
    val_rec = recall_score(val_targets, val_preds, zero_division=0)
    val_f1 = f1_score(val_targets, val_preds, zero_division=0)
    val_history.append((val_acc, val_prec, val_rec, val_f1))

    print(f"Validation Metrics [Epoch {epoch+1}]:Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}, F1 Score: {val_f1:.4f}")
    
     # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "resnet3d_model.pt") # change depending on face features or brain tissue
        print(f"Best model saved at epoch {epoch+1} with F1 Score: {val_f1:.4f}")


Epoch [1/5], Step [1/1912], Loss: 0.5685
Epoch [1/5], Step [2/1912], Loss: 0.4429
Epoch [1/5], Step [3/1912], Loss: 0.3527
Epoch [1/5], Step [4/1912], Loss: 0.1843
Epoch [1/5], Step [5/1912], Loss: 0.1546
Epoch [1/5], Step [6/1912], Loss: 0.7048
Epoch [1/5], Step [7/1912], Loss: 0.3626
Epoch [1/5], Step [8/1912], Loss: 1.4671
Epoch [1/5], Step [9/1912], Loss: 0.4019
Epoch [1/5], Step [10/1912], Loss: 0.3591
Epoch [1/5], Step [11/1912], Loss: 0.1816
Epoch [1/5], Step [12/1912], Loss: 0.3226
Epoch [1/5], Step [13/1912], Loss: 0.2121
Epoch [1/5], Step [14/1912], Loss: 0.2173
Epoch [1/5], Step [15/1912], Loss: 0.6635
Epoch [1/5], Step [16/1912], Loss: 0.1608
Epoch [1/5], Step [17/1912], Loss: 0.1325
Epoch [1/5], Step [18/1912], Loss: 0.1457
Epoch [1/5], Step [19/1912], Loss: 0.2079
Epoch [1/5], Step [20/1912], Loss: 0.1435
Epoch [1/5], Step [21/1912], Loss: 0.1210
Epoch [1/5], Step [22/1912], Loss: 0.0995
Epoch [1/5], Step [23/1912], Loss: 0.1589
Epoch [1/5], Step [24/1912], Loss: 0.1009
E

In [5]:
model.eval()
val_preds = []
val_targets = []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        preds = (outputs > 0.5).float()
        val_preds.extend(preds.cpu().numpy())
        val_targets.extend(labels.cpu().numpy())

acc = accuracy_score(val_targets, val_preds)
prec = precision_score(val_targets, val_preds, zero_division=0)
rec = recall_score(val_targets, val_preds, zero_division=0)
f1 = f1_score(val_targets, val_preds, zero_division=0)

print(f"Validation Metrics:Accuracy: {acc:.4f}Precision: {prec:.4f}Recall: {rec:.4f}F1 Score: {f1:.4f}")


acc = accuracy_score(targets, predictions)
prec = precision_score(targets, predictions, zero_division=0)
rec = recall_score(targets, predictions, zero_division=0)
f1 = f1_score(targets, predictions, zero_division=0)

print(f"Evaluation Metrics:Accuracy: {acc:.4f}Precision: {prec:.4f}Recall: {rec:.4f}F1 Score: {f1:.4f}")


KeyboardInterrupt: 

In [7]:
# Plot training and validation F1-score
train_f1s = [t[3] for t in train_history]
val_f1s = [v[3] for v in val_history]

plt.figure(figsize=(10, 5))
plt.plot(train_f1s, label='Train F1')
plt.plot(val_f1s, label='Val F1')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Training vs Validation F1 Score')
plt.legend()
plt.grid(True)
plt.savefig("f1_score_plot.png")
plt.show()

# Test set evaluation
model.eval()
test_preds = []
test_targets = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        preds = (outputs > 0.5).float()
        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(labels.cpu().numpy())

# Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(test_targets, test_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Test Set")
plt.savefig("confusion_matrix_face_fatures.png")
plt.show()

acc = accuracy_score(test_targets, test_preds)
prec = precision_score(test_targets, test_preds, zero_division=0)
rec = recall_score(test_targets, test_preds, zero_division=0)
f1 = f1_score(test_targets, test_preds, zero_division=0)

print(f"Test Set Metrics:Accuracy: {acc:.4f}Precision: {prec:.4f}Recall: {rec:.4f}F1 Score: {f1:.4f}")

# Save the trained model
torch.save(model.state_dict(), "resnet3d_model.pt")


NameError: name 'test_targets' is not defined

In [19]:
def run_gradcam(model, sample_img, target_layer):
    import torch.nn.functional as F
    from scipy.ndimage import zoom
    import numpy as np

    input_tensor = sample_img.unsqueeze(0).cuda().requires_grad_(True)

    class FeatureExtractor:
        def __init__(self, model, target_layer):
            self.model = model
            self.target_layer = target_layer
            self.gradients = None
            self.activations = None
            self.hook()

        def hook(self):
            def forward_hook(module, input, output):
                self.activations = output.detach()
            def backward_hook(module, grad_input, grad_output):
                self.gradients = grad_output[0].detach()
            self.target_layer.register_forward_hook(forward_hook)
            self.target_layer.register_backward_hook(backward_hook)

    # Register hooks and run forward/backward pass
    target = FeatureExtractor(model, target_layer)
    output = model(input_tensor)
    model.zero_grad()
    output.backward()

    # Compute Grad-CAM
    weights = target.gradients.mean(dim=(2, 3, 4), keepdim=True)
    cam = F.relu((weights * target.activations).sum(1)).squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    # Resize CAM to match input shape
    original_shape = sample_img.squeeze().cpu().numpy().shape
    cam_resized = zoom(cam, np.array(original_shape) / np.array(cam.shape), order=1)

    return cam_resized

def plot_multiview_overlay(original_img, cam, output_file):
    axial = min(cam.shape[0] - 1, original_img.shape[0] // 2)
    coronal = min(cam.shape[1] - 1, original_img.shape[1] // 2)
    sagittal = min(cam.shape[2] - 1, original_img.shape[2] // 2)

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(original_img[:, axial, :], cmap='gray')
    axs[0].imshow(cam[axial], cmap='hot', alpha=0.5)
    axs[0].set_title("Axial")

    axs[1].imshow(original_img[coronal], cmap='gray')
    axs[1].imshow(cam[:, coronal, :], cmap='hot', alpha=0.5)
    axs[1].set_title("Coronal")

    axs[2].imshow(original_img[:, :, sagittal], cmap='gray')
    axs[2].imshow(cam[:, :, sagittal], cmap='hot', alpha=0.5)
    axs[2].set_title("Sagittal")

    plt.suptitle(output_file)
    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()

In [ ]:
# Facial feature detection model
model_face = ResNet3D()
model_face.load_state_dict(torch.load("best_resnet3d_model.pt"))
model_face.eval()

# Brain tissue loss detection model
model_tissue = ResNet3D()
model_tissue.load_state_dict(torch.load("resnet3d_brain_tissue_model.pt"))
model_tissue.eval()

sample_img, _ = test_dataset[12] # 4, 8
original = sample_img.squeeze().cpu().numpy()

# Grad-CAM for facial feature model
cam_face = run_gradcam(model_face.cuda(), sample_img, model_face.layer4[-1].conv2)
plot_multiview_overlay(original, cam_face, "gradcam_face_multiview.png")

# Grad-CAM for tissue loss model
cam_tissue = run_gradcam(model_tissue.cuda(), sample_img, model_tissue.layer4[-1].conv2)
plot_multiview_overlay(original, cam_tissue, "gradcam_tissue_multiview.png")